In [1]:
#%% Importing Libraries
import pandas as pd , numpy as np
pd.set_option('max_columns',200)
pd.set_option('max_rows',200)

import pickle , warnings
warnings.filterwarnings('ignore')

from scipy.optimize import minimize , Bounds , LinearConstraint , NonlinearConstraint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Media

In [2]:
def intial_manipulations(path , optimization = 0):
    
    #reading data
    df = pd.read_csv(path).round(2)
    
    # getting inputs from user
    total_grp = (df['input_weekly_grp'] * df['input_WOA']).max()
    total_spend = df['input_spends'].max()

    ####################################################################################################
    # for mat 3 min cons
    current_volume = 75.3e6 # database
    growth_ambition_perc = 0.1 ## user_input
    growth_ambition_volume = current_volume * growth_ambition_perc
    current_volume_contribution = (df['current_metric_value'] * df['current_effectiveness']).sum() 

    matrix_3_min_cons = growth_ambition_volume + current_volume_contribution
    ####################################################################################################
    
    df_only = df[df['genre_platform'] != 'Total']
    df_total = df[df['genre_platform'] == 'Total']
    
    if optimization == 1:
    
        def bounds_col(value , media_type , min_ = 0 , max_ = 0 , total_grp = 0 , total_spends = 0):

            if min_ != 0:
                if media_type == 'TV': return 0.2 * value
                if media_type == 'Digital': return 0.05 * value

            if max_ != 0:
                if media_type == 'TV': return 0.7 * total_grp
                if media_type == 'Digital': return (0.3 * total_spends) / value

        def cost(value_to_optimize , data  = df_only):  

            data['Recommended_val'] = value_to_optimize
            cal_values = list(map(lambda rv , mi, ma : True if mi <= rv <= ma else False , data['Recommended_val'] ,  data['lower_bounds'] ,  data['upper_bounds']))
            if all(cal_values): total_val = (data['input_cost_per_metric'] * (data['Recommended_val'] - data['current_metric_value'])).sum()
            else: total_val = 1e9

            return total_val

        # getting lower and upper bounds
        df_only['lower_bounds'] = tuple(map(lambda x , y : bounds_col(x,y , min_ = 1) , df_only['current_metric_value'] , df_only['media_type']))
        df_only['upper_bounds'] = tuple(map(lambda x , y : bounds_col(x,y , max_ = 1 , total_grp = total_grp , total_spends = total_spend) ,
                                       df_only['input_cost_per_metric'] , df_only['media_type']))

        # making contraint matrix
        df_only['matrix_row_1_tv'] = df_only['media_type'].apply(lambda x : 1 if x == 'TV' else 0)
        df_only['matrix_row_2_digital'] = df_only['media_type'].apply(lambda x : 1 if x == 'Digital' else 0)
        df_only['matrix_row_3_vg'] = df_only['current_effectiveness'].copy()

        # random start values
        df_only['start_values'] = list(map(lambda x , y : np.random.randint(x , y) , df_only['lower_bounds'] , df_only['upper_bounds'].replace(0 , 1)))

        # bounds and constraint
        bounds = Bounds( df_only['lower_bounds'] , df_only['upper_bounds'] , keep_feasible = False)
        cons_ = LinearConstraint(df_only.filter(regex = 'matrix').T.values , 
                                 [0, 0 , matrix_3_min_cons], [total_grp, total_spend, np.inf])


        # optimization
        optimize_output = minimize(cost , x0 = df_only['start_values'] , args = df_only ,
                                   bounds = bounds  ,  method = 'trust-constr' , constraints = cons_ ,
                                   hess = lambda x , data : np.zeros((x.shape[0], x.shape[0])) ,
                                   options = {'maxiter' : 1000 , 'disp' : True} )

    return locals()

In [3]:
all_data_mani = intial_manipulations(path = r"..\CSV Data as Input\media_records (1).csv" , optimization = 1)

all_data_mani.keys()

df_only = all_data_mani['df_only'].reset_index(drop = True)
df_total = all_data_mani['df_total'].reset_index(drop = True)

`xtol` termination condition is satisfied.
Number of iterations: 184, function evaluations: 787, CG iterations: 172, optimality: 4.14e+02, constraint violation: 8.94e+07, execution time:  2.2 s.


dict_keys(['path', 'optimization', 'df', 'current_volume', 'growth_ambition_perc', 'growth_ambition_volume', 'current_volume_contribution', 'matrix_3_min_cons', 'df_only', 'df_total', 'cost', 'bounds', 'cons_', 'optimize_output', 'bounds_col', 'total_grp', 'total_spend'])

In [4]:
def brand_level(_data , _total_data , val): 
    data_ = _data.copy()
    cols = ['country' , 'year' , 'timeline' , 'analysis_period' , 'category' , 'brand' , 'media_type']
    
    data_ = data_.drop('id',1).groupby(cols, sort = False).sum()
    _total_data[val] = data_.reset_index()[val]
    
    return data_

def spends_output(data , total_data ,try_ = 0):
    if try_ == 0: 
        data['Recommended_spends'] = data['Recommended_val'] * data['input_cost_per_metric']
        brand_level(data , total_data , 'Recommended_spends')
        
    if try_ == 1:
        data['Try_spends_output'] = data['Try_scenario_input'] * data['input_cost_per_metric'] # ask honey to provide impressions for digital
        brand_level(data , total_data , 'Try_spends_output')
        
def volume_contribution(data , total_data ,try_ = 0):
    if try_ == 0: 
        data['Recommended_vc'] = data['Recommended_val'] * data['current_effectiveness']
        brand_level(data , total_data , 'Recommended_vc')
        
    if try_ == 1: 
        data['Try_vc_output'] = tuple(map(lambda grp , eff , med , sp , cpm : grp * eff if me == 'TV'
                                          else (sp / cpm) * eff if med == 'Digital' else 0),
                                      data['Try_tv_grp'] , data['current_effectiveness'] ,data['media_type'] ,
                                      data['Try_scenerio_spends_digital'] , data['input_cost_per_metric'])
        brand_level(data , total_data , 'Try_vc_output')
        
def roi_output(data ,total_df ,  try_ = 0):
    if try_ == 0: 
        data['Recommended_roi'] = (data['Recommended_vc'] * data['current_ASP']) / data['Recommended_spends']
        
        # brand level roi
        data['Recommended_roi_brand'] = data['Recommended_roi'] * data['Recommended_spends']
        brand_level(data , total_df , 'Recommended_roi_brand')
        total_df['Recommended_roi_brand'] = total_df['Recommended_roi_brand'] / total_df['Recommended_spends']
        total_df.rename(columns = {'Recommended_roi_brand' : 'Recommended_roi'} , inplace = True)
        data.drop('Recommended_roi_brand' , 1 , inplace = True)
        
    if try_ == 1: 
        data['Try_roi_output'] = (data['Try_vc'] * data['current_ASP']) / data['Try_spends_output']
        
        data['Try_roi_brand_output'] = data['Try_roi_output'] * data['Try_spends_output']
        brand_level(data , total_df , 'Try_roi_brand_output')
        total_df['Try_roi_brand_output'] = total_df['Try_roi_brand_output'] / total_df['Try_spends_output']
        total_df.rename(columns = {'Try_roi_brand_output' : 'Try_roi_output'} , inplace = True)
        data.drop('Try_roi_brand_output' , 1 , inplace = True)

In [5]:
spends_output(df_only , df_total)
volume_contribution(df_only, df_total)
roi_output(df_only , df_total)

In [6]:
final_df = pd.concat([df_only , df_total])

In [7]:
final_df

,id,country,year,timeline,analysis_period,category,brand,media_type,genre_platform,current_spends,current_metric_value,current_effectiveness,current_impressions,current_volume,current_ASP,current_revenue,current_ROI,input_cost_per_metric,input_weekly_grp,input_WOA,input_spends,input_operating_point,metric_type,currency,spends_divisor,lower_bounds,upper_bounds,matrix_row_1_tv,matrix_row_2_digital,matrix_row_3_vg,start_values,Recommended_val,Recommended_spends,Recommended_vc,Recommended_roi
0,2,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,TV,Entertainment,860376.0,1456,63.00,NaN,91728.00,9,837093.28,1.0,591.00,NaN,NaN,NaN,NaN,grp,usd,1000,291.20,1.680000e+04,1.0,0.0,63.00,1200.0,2.912000e+02,1.720992e+05,1.834560e+04,0.959391
1,3,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,TV,Sports Bein,78788.0,0,71.00,NaN,0.00,9,0.00,NaN,73.00,NaN,NaN,NaN,NaN,grp,usd,1000,0.00,1.680000e+04,1.0,0.0,71.00,15156.0,1.543502e+04,1.126756e+06,1.095886e+06,8.753425
2,4,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,TV,Sports Non-Bein,499090.0,1332,195.00,NaN,259740.00,9,2370340.66,4.7,375.00,NaN,NaN,NaN,NaN,grp,usd,1000,266.40,1.680000e+04,1.0,0.0,195.00,15348.0,1.572393e+04,5.896475e+06,3.066167e+06,4.680000
3,5,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,TV,Movies & Series,601441.0,1201,136.00,NaN,163336.00,9,1490575.04,2.5,501.00,NaN,NaN,NaN,NaN,grp,usd,1000,240.20,1.680000e+04,1.0,0.0,136.00,9662.0,9.398945e+03,4.708871e+06,1.278257e+06,2.443114
4,8,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,anghami.com,NaN,0,0.07,NaN,0.00,9,0.00,NaN,0.25,NaN,NaN,NaN,NaN,Completed Views,usd,1000,0.00,3.000000e+06,0.0,1.0,0.07,937336.0,9.371938e+05,2.342985e+05,6.560357e+04,2.520000
5,9,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,Facebook,296127.0,4830586,0.02,NaN,77289.38,9,705328.98,2.4,0.06,NaN,NaN,NaN,NaN,Completed Views,usd,1000,241529.30,1.250000e+07,0.0,1.0,0.02,7069503.0,7.069361e+06,4.241617e+05,1.413872e+05,3.000000
6,10,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,Instagram,108441.0,1004215,0.02,NaN,18075.87,9,164957.15,1.5,0.11,NaN,NaN,NaN,NaN,Completed Views,usd,1000,50210.75,6.818182e+06,0.0,1.0,0.02,1255888.0,1.255746e+06,1.381320e+05,2.511492e+04,1.636364
7,11,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,Programmatic,144638.0,4227535,0.02,NaN,80323.16,9,733014.80,5.1,0.03,NaN,NaN,NaN,NaN,Completed Views,usd,1000,211376.75,2.500000e+07,0.0,1.0,0.02,12378271.0,1.237813e+07,3.713439e+05,2.475626e+05,6.000000
8,12,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,Resolution Media,192094.0,6017385,0.11,NaN,631825.43,9,5765925.52,30.0,0.03,NaN,NaN,NaN,NaN,Completed Views,usd,1000,300869.25,2.500000e+07,0.0,1.0,0.11,18373145.0,1.837300e+07,5.511901e+05,2.021030e+06,33.000000
9,13,KSA,2021,MAT May 2021,Jan 2018 to May 2021,Beverages,Pepsi,Digital,Snapchat,208919.0,1976612,0.00,NaN,7906.45,9,72152.83,0.3,0.11,NaN,NaN,NaN,NaN,Completed Views,usd,1000,98830.60,6.818182e+06,0.0,1.0,0.00,1615791.0,1.615649e+06,1.777214e+05,0.000000e+00,0.000000


### Distribution

In [9]:
recommended_volume_contribution = (final_df['Recommended_val'] * final_df['current_effectiveness']).sum()

current_volume = 75.3e6 # database
growth_ambition_perc = 0.1 ## user_input
growth_ambition_volume = current_volume * growth_ambition_perc
current_volume_contribution = (final_df['current_metric_value'] * final_df['current_effectiveness']).sum() 

achieved_from_media = recommended_volume_contribution - current_volume_contribution

In [10]:
# current_volume = 75.3e6 # database
# growth_ambition_perc = 0.1 ## user_input
# growth_ambition_volume = current_volume*growth_ambition_perc

remainder_volume_growth =  growth_ambition_volume - achieved_from_media # 124

In [11]:
# data manipulation
df_dist = pd.read_excel(r"..\CSV Data as Input\KSA Pepsi Distribution.xlsx" )

dfs, lever, current_val = [] , ['price' , 'distribution' , 'trade'] , ['current_price_per_pack', 'current_distribution', 'current_trade']
for lev , cv in zip(lever , current_val):
    op = df_dist[['id','current_volume','current_price_per_volume', cv, lev+'_elasticity','channel']].rename(columns = {cv : 'current_val' , lev + '_elasticity' : 'elasticity'}).dropna()
    op['lever'] = lev
    dfs.append(op)    

df_dist_analysis = pd.concat(dfs).reset_index(drop = True)

def volume_growth(data, return_sum = False , try_ = 0):
    if try_ == 0: pct_change_lever = (data['Recommended_val'] / data['current_val']) - 1
    if try_ == 1: pct_change_lever = (data['Try_input'] / data['current_val']) - 1
    pct_change_volume = pct_change_lever * data['elasticity']
    vg = pct_change_volume * data['current_volume']
    if return_sum: return sum(vg)
    return vg

def volume_constraint(x , data = df_dist_analysis.copy()):
    data['Recommended_val'] = x
    vg = volume_growth(data, return_sum=True)
    return vg

def cost_of_distribution(data):
    cost = 0.01 * (data['Recommended_val'] - data['current_val'])
    return sum(cost)

def cost_of_price(data):
    vg = volume_growth(data)
    cost = vg * (data['Recommended_val'] - data['current_val'])
    return sum(cost)

def cost_of_trade(data):
    vg = volume_growth(data)
    cost = vg * (data['Recommended_val'] - data['current_val']) * data['current_price_per_volume']
    return sum(cost)

def cost(values_to_optimize , data):
    
    data['Recommended_val'] = values_to_optimize
    cal_values = list(map(lambda rv , mi, ma : True if mi <= rv <= ma else False , data['Recommended_val'] ,  data['lower_bounds'] ,  data['upper_bounds']))
    
    if all(cal_values):
        
        p = cost_of_price(data[data['lever'] == 'price'])
        d = cost_of_distribution(data[data['lever'] == 'distribution'])
        t = cost_of_trade(data[data['lever'] == 'trade'])
    
        total_val = p + d + t
        return total_val
       
    else: 
        total_val = 1e9

    return total_val

# Bounds
B = 0.10
df_dist_analysis['lower_bounds'] = df_dist_analysis['current_val'] * (1 - B)
df_dist_analysis['upper_bounds'] = df_dist_analysis['current_val'] * (1 + B)

bounds = Bounds(lb = df_dist_analysis['lower_bounds'], ub = df_dist_analysis['upper_bounds'], keep_feasible = False)

nl_constraints = NonlinearConstraint(volume_constraint,
                                     lb = remainder_volume_growth ,
                                     ub = np.inf, 
                                     keep_feasible = False)

df_dist_analysis['start_values'] = list(map(lambda x , y : np.arange(x,y,0.01)[np.random.randint(len(np.arange(x,y,0.01)))] ,
                                            df_dist_analysis['lower_bounds'] , df_dist_analysis['upper_bounds']))

iterations = 2#int(1e3)

In [12]:
result = minimize(cost , x0 = df_dist_analysis['start_values'] , args = df_dist_analysis, 
                  bounds = bounds , constraints = [nl_constraints] , hess = lambda x , data : np.zeros((x.shape[0], x.shape[0])),
                  method = 'trust-constr' , 
                  options = {'maxiter': iterations, 'verbose': 1, 'factorization_method':'SVDFactorization'})
                  
print(result.success, result.cg_stop_cond)


def try_sc(data , try_ = 0):
    if try_sc == 1: out = volume_growth(data , try_ = 1)
    return vg

def out(lv , val):
    if lv == 'price': return (val , np.nan , np.nan)
    if lv == 'distribution': return (np.nan, val , np.nan)
    if lv == 'trade': return (np.nan , np.nan , val)

The maximum number of function evaluations is exceeded.
Number of iterations: 2, function evaluations: 80, CG iterations: 1, optimality: 1.96e+06, constraint violation: 2.20e+06, execution time: 0.16 s.
False 2


In [13]:
new_names = ['Recommended_' + i.replace('current_','') for i in current_val]
df_dist_analysis[new_names] = tuple(map(
    lambda lv , val : out(lv ,val) , df_dist_analysis['lever'] , df_dist_analysis['Recommended_val'] ))

df_dist_analysis.drop(columns = ['Recommended_val',], inplace = True)

In [14]:
df_dist_analysis

,id,current_volume,current_price_per_volume,current_val,elasticity,channel,lever,lower_bounds,upper_bounds,start_values,Recommended_price_per_pack,Recommended_distribution,Recommended_trade
0,1,30549,42.228524,1.784986,-7.920279,Supermarket,price,1.606487,1.963484,1.896487,1.892064,NaN,NaN
1,2,1494,44.021514,4.396876,-7.920279,Supermarket,price,3.957188,4.836563,4.187188,4.201009,NaN,NaN
2,3,57298,45.388873,1.998466,-7.920279,Supermarket,price,1.798619,2.198313,1.838619,1.848393,NaN,NaN
3,4,128232,44.191023,2.845964,-0.807162,Supermarket,price,2.561368,3.130561,2.641368,2.654598,NaN,NaN
4,5,946158,39.375545,16.177799,-0.807162,Supermarket,price,14.560019,17.795579,17.050019,17.046081,NaN,NaN
5,6,1205094,39.628811,9.029592,-0.807162,Supermarket,price,8.126633,9.932551,8.476633,8.493064,NaN,NaN
6,7,1692112,42.902077,3.053779,-0.807162,Supermarket,price,2.748401,3.359157,3.318401,3.317001,NaN,NaN
7,8,30895,47.300982,3.899114,-0.807162,Supermarket,price,3.509203,4.289025,3.629203,3.646933,NaN,NaN
8,9,939269,25.682033,4.645755,-0.997443,Supermarket,price,4.181179,5.110330,4.771179,4.767574,NaN,NaN
9,10,2026107,18.939836,8.345761,-1.677083,Supermarket,price,7.511185,9.180337,8.881185,8.890925,NaN,NaN


In [15]:
merge_cols = ['id','channel','current_volume','current_price_per_volume']

In [16]:
len(df_dist)
len(df_dist_analysis)

48

78

In [17]:
final_exec = pd.merge(df_dist, df_dist_analysis[new_names + ['id','current_volume','current_price_per_volume']] ,
                      on = ['id','current_volume','current_price_per_volume'] , how = 'inner')

In [18]:
len(final_exec)

78

In [19]:
final_exec

,id,country,channel,current_metrics_time,current_volume_time,analysis_period,category,brand,currency,volume_unit,pack_name,current_volume,price_elasticity,current_price_per_volume,current_price_per_pack,current_volume_per_pack,distribution_type,current_distribution,distribution_elasticity,current_trade,trade_elasticity,Recommended_price_per_pack,Recommended_distribution,Recommended_trade
0,1,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 240ML,30549,-7.920279,42.228524,1.784986,0.042270,NaN,NaN,NaN,NaN,NaN,1.892064,NaN,NaN
1,2,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 245ML,1494,-7.920279,44.021514,4.396876,0.099880,NaN,NaN,NaN,NaN,NaN,4.201009,NaN,NaN
2,3,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 250ML,57298,-7.920279,45.388873,1.998466,0.044030,NaN,NaN,NaN,NaN,NaN,1.848393,NaN,NaN
3,4,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 320ML,128232,-0.807162,44.191023,2.845964,0.064401,NaN,NaN,NaN,NaN,NaN,2.654598,NaN,NaN
4,5,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 325ML,946158,-0.807162,39.375545,16.177799,0.410859,NaN,NaN,NaN,NaN,NaN,17.046081,NaN,NaN
5,6,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 330ML,1205094,-0.807162,39.628811,9.029592,0.227854,NaN,NaN,NaN,NaN,NaN,8.493064,NaN,NaN
6,7,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 355ML,1692112,-0.807162,42.902077,3.053779,0.071180,NaN,NaN,NaN,NaN,NaN,3.317001,NaN,NaN
7,8,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,CAN 360ML,30895,-0.807162,47.300982,3.899114,0.082432,NaN,NaN,NaN,NaN,NaN,3.646933,NaN,NaN
8,9,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,MSPET 1L,939269,-0.997443,25.682033,4.645755,0.180895,NaN,NaN,NaN,NaN,NaN,4.767574,NaN,NaN
9,10,KSA,Supermarket,2021-05-01,CY 2020,Jan 2018 to May 2021,Beverage,Pepsi,SAR,8Oz case,MSPET 2.2L,2026107,-1.677083,18.939836,8.345761,0.440646,NaN,NaN,NaN,NaN,NaN,8.890925,NaN,NaN
